# Installations

In [ ]:
# %pip install -r "Requirements/Pbr_requirements.txt" # Plumber wale k liye hai ye

In [ ]:
# %pip install -r "Requirements/crop_requirments.txt" 
# %pip install -r "Requirements/recog_requirments.txt"  
# %pip install -r "Requirements/to_Img_requirments.txt"

# OR

Installation Function to Page wise Image Conversion

In [ ]:
# %pip install pymupdf

Installation Function to Cropping Tables

In [ ]:
# %%capture
# %pip install "unstructured[all-docs]"

installations for Function to Recognise and format tables as csv

In [ ]:
# %pip install paddlepaddle paddleocr tensorflow opencv-python pandas

# Imports

In [ ]:
import os
# import pdfplumber
import pandas as pd
# import fitz
import cv2
import numpy as np
import tensorflow as tf
from paddleocr import PaddleOCR
#from unstructured.partition.image import partition_image

# Define Directories

In [ ]:
input_dir = "INPUTS/PDFs"
output_dir = "RESULTS"
deb_output_dir = "DEBUG_IMGs"

# PyMuPdf Image Extraction

In [ ]:
for filename in os.listdir(input_dir):
    if filename.lower().endswith(".pdf"):
        # Build the full path to the PDF file
        pdf_path = os.path.join(input_dir, filename)
        # Extract the base name (e.g., 'document' from 'document.pdf')
        pdf_name = os.path.splitext(filename)[0]

        # Create the corresponding output directory
        output_folder = f"RESULTS/IMAGES/{pdf_name}-images"
        os.makedirs(output_folder, exist_ok=True)

        # Open the PDF file
        pdf_document = fitz.open(pdf_path)
        print(f"Processing PDF: {pdf_path}")

        # Iterate over each page in the PDF
        for page_number in range(pdf_document.page_count):
            page = pdf_document[page_number]
            images = page.get_images(full=True)  # Get all images on the page
            print(f"  Page {page_number + 1} has {len(images)} image(s).")

            # Extract and save each image
            for img_index, img in enumerate(images):
                xref = img[0]  # Reference number of the image
                base_image = pdf_document.extract_image(xref)  # Extract image data
                image_bytes = base_image["image"]  # Raw image bytes

                # Construct a filename for each image
                image_path = os.path.join(
                    output_folder,
                    f"Page_{page_number + 1}_Image_{img_index + 1}.png"
                )

                # Save the image to a file
                with open(image_path, "wb") as img_file:
                    img_file.write(image_bytes)
                print(f"    Saved: {image_path}")

        # Close the PDF document
        pdf_document.close()

# Pdfplumber Tables Extraction

In [ ]:
pdf_files = [f for f in os.listdir(input_dir) if f.endswith(".pdf")] # Get a list of all PDF files

for pdf_file in pdf_files:
    input_path = os.path.join(input_dir, pdf_file)  # Full path to the current PDF
    pdf_name = os.path.splitext(os.path.basename(pdf_file))[0]  # Extract PDF name without extension
    
    output_base_dir1 = f"TABLES/{pdf_name}-tables"  # Tables output directory
    debug_output_dir1 = f"{pdf_name}-Markings"  # Debug output directory
    output_base_dir = os.path.join(output_dir, output_base_dir1)
    debug_output_dir = os.path.join(deb_output_dir, debug_output_dir1)
    
    os.makedirs(output_base_dir, exist_ok=True)
    os.makedirs(debug_output_dir, exist_ok=True)

    print(f"Processing file: {pdf_file}")
    
    with pdfplumber.open(input_path) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            print(f"Processing Page {i} of {pdf_file}")
            
            # Debug table detection visually
            debug_image = page.to_image()
            debug_image.draw_rects(page.debug_tablefinder().edges)
            debug_image_path = os.path.join(debug_output_dir, f"Page_{i}_debug.png")
            debug_image.save(debug_image_path)
            print(f"Debug image for Page {i} saved to {debug_image_path}")
            
            # Extract tables from the page
            tables = page.extract_tables()
            if tables:
                for j, table in enumerate(tables, start=1):
                    df = pd.DataFrame(table[1:], columns=table[0])  # Assuming first row as header
                    output_csv = os.path.join(output_base_dir, f"Page_{i}_Table_{j}.csv") #Output file path for each table
                    df.to_csv(output_csv, index=False)  # Save the DataFrame to a CSV file
                    print(f"Table {j} from Page {i} saved to {output_csv}")
            else:
                print(f"No tables found on Page {i} of {pdf_file}.")

print("All files have been successfully processed.")

# Pdfplumber Page wise Text Extraction

In [ ]:
for pdf_file in os.listdir(input_dir):
    if pdf_file.endswith(".pdf"):
        pdf_path = os.path.join(input_dir, pdf_file)
        pdf_name = os.path.splitext(pdf_file)[0]  # Get the PDF file name without extension

        text_output_folder = f"RESULTS/PAGE_TEXTS/{pdf_name}-texts"
        image_output_folder = f"DEBUG_IMGs/{pdf_name}-annotations"
        os.makedirs(text_output_folder, exist_ok=True)
        os.makedirs(image_output_folder, exist_ok=True)

        with pdfplumber.open(pdf_path) as pdf:

            for page_number, page in enumerate(pdf.pages):
                # Extract table bounding boxes
                tables = page.find_tables()
                table_bboxes = [table.bbox for table in tables]

                # Extract all characters with positions
                characters = page.chars

                # Filter out characters that overlap with table bounding boxes
                non_table_chars = [
                    char for char in characters
                    if not any(
                        bbox[0] <= float(char["x0"]) <= bbox[2]
                        and bbox[1] <= float(char["top"]) <= bbox[3]
                        for bbox in table_bboxes
                    )
                ]

                # Sort characters by their y-coordinate (top) and x-coordinate (left-to-right)
                non_table_chars.sort(key=lambda c: (float(c["top"]), float(c["x0"])))

                # Reconstruct the text layout, character by character
                reconstructed_text = ""
                last_top = None
                last_x1 = None
                line_buffer = []

                # Thresholds
                new_line_threshold = 5   # vertical distance to consider it a new line
                space_gap_threshold = 1  # horizontal gap to consider adding a space

                for char in non_table_chars:
                    current_top = float(char["top"])
                    current_x0 = float(char["x0"])
                    current_x1 = float(char["x1"])
                    text_char = char["text"]

                    # Detect new line based on the vertical position
                    if last_top is None or abs(current_top - last_top) > new_line_threshold:
                        # Finish the existing line buffer
                        if line_buffer:
                            reconstructed_text += "".join(line_buffer) + "\n"
                        # Start a new line buffer with the current character
                        line_buffer = [text_char]
                        last_top = current_top
                        last_x1 = current_x1
                    else:
                        # Same line, decide if we need to add a space for a big gap
                        if last_x1 is not None:
                            gap = current_x0 - last_x1

                            # If gap is large enough and the next character isn't an explicit space,
                            # add a single space (but do NOT double-space if we already ended with a space)
                            if gap > space_gap_threshold and text_char not in [" ", "\t"]:
                                if not line_buffer or (line_buffer and line_buffer[-1] != " "):
                                    line_buffer.append(" ")

                        # Now add the actual character
                        line_buffer.append(text_char)
                        last_x1 = current_x1
                        last_top = current_top

                # Add the final line to the reconstructed text
                if line_buffer:
                    reconstructed_text += "".join(line_buffer) + "\n"

                # Save the reconstructed text to a file
                text_file_path = os.path.join(text_output_folder, f"page{page_number + 1}.txt")
                with open(text_file_path, "w", encoding="utf-8") as text_file:
                    text_file.write(reconstructed_text)

                print(f"Character-level text with spacing from Page {page_number + 1} saved to: {text_file_path}")

                # Annotate the remaining characters (non-table chars)
                page_image = page.to_image()
                annotated_image = page_image.reset().draw_rects([
                    {
                        "x0": float(char["x0"]),
                        "x1": float(char["x1"]),
                        "top": float(char["top"]),
                        "bottom": float(char["bottom"]),
                    }
                    for char in non_table_chars
                ])

                # Save the annotated image
                annotated_image_path = os.path.join(image_output_folder, f"page_{page_number + 1}_annotated.png")
                annotated_image.save(annotated_image_path)
                print(f"Annotated image saved to: {annotated_image_path}")

        print(f"Processing complete for {pdf_name}. Check the folders for results.")

# Unstructured Tables Extraction (OCR Based)

Function to Page wise Image Conversion

In [ ]:
def extract_pdf_pages(input_dir, output_base_folder="INPUTS\PDF IMAGEs\Pages", dpi=300):


    os.makedirs(output_base_folder, exist_ok=True)
    pdf_files = [f for f in os.listdir(input_dir) if f.lower().endswith('.pdf')]

    for pdf_file in pdf_files:
        pdf_path = os.path.join(input_dir, pdf_file)
        pdf_name = os.path.splitext(pdf_file)[0]
        pdf_output_folder = os.path.join(output_base_folder, f"{pdf_name}-Pages")
        os.makedirs(pdf_output_folder, exist_ok=True)

        pdf_document = fitz.open(pdf_path)
        page_count = pdf_document.page_count
        print(f"Processing '{pdf_file}' ({page_count} pages)...")

        for page_number in range(page_count):
            page = pdf_document[page_number]
            pix = page.get_pixmap(dpi=dpi, colorspace=fitz.csRGB)  # Render at specified DPI in RGB
            image_path = os.path.join(pdf_output_folder, f'page_{page_number + 1}.png')
            pix.save(image_path)
            print(f'Saved: {image_path}')

        pdf_document.close()
        print(f"Finished processing '{pdf_file}'! Pages saved in '{pdf_output_folder}'\n")

    print("\n All PDFs processed successfully! Extracted pages are stored in '{output_base_folder}'.")

extract_pdf_pages(input_dir)

Function to Cropping Tables

In [ ]:
# from unstructured.partition.image import partition_image

def extract_and_crop_tables(input_base_folder, output_base_folder="INPUTS\PDF IMAGEs\Cropped Tables", top_left_padding=5, bottom_right_padding=7):

    os.makedirs(output_base_folder, exist_ok=True)

    subfolders = [f for f in os.listdir(input_base_folder) if os.path.isdir(os.path.join(input_base_folder, f))]

    for subfolder in subfolders:
        input_folder = os.path.join(input_base_folder, subfolder)
        output_folder = os.path.join(output_base_folder, f"{subfolder}-Cropped")

        # Ensure the output folder exists for each subfolder
        os.makedirs(output_folder, exist_ok=True)

        # Function to crop an image with padding and save it
        def crop_with_padding(image_path, coordinates, output_folder, filename):
            """Crop the specified coordinates from the image, apply padding, and save it."""
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error loading image: {image_path}")
                return

            x_min = int(min(pt[0] for pt in coordinates))
            y_min = int(min(pt[1] for pt in coordinates))
            x_max = int(max(pt[0] for pt in coordinates))
            y_max = int(max(pt[1] for pt in coordinates))

            # Apply top-left and bottom-right padding
            x_min_padded = max(0, x_min - top_left_padding)
            y_min_padded = max(0, y_min - top_left_padding)
            x_max_padded = min(image.shape[1], x_max + bottom_right_padding * 2)
            y_max_padded = min(image.shape[0], y_max + bottom_right_padding)

            # Crop the image
            cropped_image = image[y_min_padded:y_max_padded, x_min_padded:x_max_padded]

            # Save cropped image
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, cropped_image)
            print(f"Cropped image saved to: {output_path}")

        # Dictionary to count tables per image
        table_counter = {}

        # Get all image files in the input folder
        image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        # Process each image file
        for image_file in image_files:
            image_path = os.path.join(input_folder, image_file)
            image_name = os.path.splitext(image_file)[0]  # Extract image name without extension

            # Perform table detection using Unstructured's partition_image
            elements = partition_image(filename=image_path, infer_table_structure=True, strategy='hi_res')

            # Convert elements to JSON structure
            element_dict = [el.to_dict() for el in elements]

            # Extract and crop tables
            for item in element_dict:
                if isinstance(item, dict) and item.get("type") == "Table":
                    try:
                        coordinates = item["metadata"]["coordinates"]["points"]

                        # Count tables per image
                        table_counter[image_name] = table_counter.get(image_name, 0) + 1
                        table_number = table_counter[image_name]

                        # Set output filename for cropped table image
                        filename = f"{image_name}_Table_{table_number}.png"

                        # Crop and save the table
                        crop_with_padding(image_path, coordinates, output_folder, filename)

                    except KeyError as e:
                        print(f"Missing key {e} in item: {item}")

        print(f"\n Finished processing folder: {subfolder}! Cropped tables saved in: {output_folder}")

    print("\n All image folders processed successfully! Cropped tables are stored in:", output_base_folder)

input_directory = "INPUTS\PDF IMAGEs\Pages"
output_directory = "INPUTS\PDF IMAGEs\Cropped Tables"

extract_and_crop_tables(input_directory, output_directory)

Function to Recognise and format tables as csv

In [ ]:
def extract_tables_from_images(image_folder, output_folder="Extracted Tables"):

    ocr = PaddleOCR(lang='en')

    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Function to compute intersection of two bounding boxes
    def intersection(box_1, box_2):
        return [box_2[0], box_1[1], box_2[2], box_1[3]]

    # Function to compute IoU (Intersection over Union)
    def iou(box_1, box_2):
        x_1 = max(box_1[0], box_2[0])
        y_1 = max(box_1[1], box_2[1])
        x_2 = min(box_1[2], box_2[2])
        y_2 = min(box_1[3], box_2[3])

        inter = abs(max((x_2 - x_1, 0)) * max((y_2 - y_1), 0))
        if inter == 0:
            return 0

        box_1_area = abs((box_1[2] - box_1[0]) * (box_1[3] - box_1[1]))
        box_2_area = abs((box_2[2] - box_2[0]) * (box_2[3] - box_2[1]))

        return inter / float(box_1_area + box_2_area - inter)

    # Get all image files from the folder
    image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Process each image
    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)
        image_cv = cv2.imread(image_path)

        # Ensure the image was loaded correctly
        if image_cv is None:
            print(f"Error loading image: {image_path}")
            continue

        image_height, image_width = image_cv.shape[:2]

        # Perform OCR
        output = ocr.ocr(image_path)[0]

        # Extract bounding boxes, detected text, and confidence scores
        boxes = [line[0] for line in output]
        texts = [line[1][0] for line in output]
        probabilities = [line[1][1] for line in output]

        # Copy image for processing
        im = image_cv.copy()

        horiz_boxes = []
        vert_boxes = []

        # Generate horizontal and vertical bounding boxes
        for box in boxes:
            x_h, x_v = 0, int(box[0][0])
            y_h, y_v = int(box[0][1]), 0
            width_h, width_v = image_width, int(box[2][0] - box[0][0])
            height_h, height_v = int(box[2][1] - box[0][1]), image_height

            horiz_boxes.append([x_h, y_h, x_h + width_h, y_h + height_h])
            vert_boxes.append([x_v, y_v, x_v + width_v, y_v + height_v])

            cv2.rectangle(im, (x_h, y_h), (x_h + width_h, y_h + height_h), (0, 0, 255), 1)
            cv2.rectangle(im, (x_v, y_v), (x_v + width_v, y_v + height_v), (0, 255, 0), 1)

        # Apply Non-Maximum Suppression (NMS) for horizontal boxes
        horiz_out = tf.image.non_max_suppression(
            horiz_boxes, probabilities, max_output_size=1000, iou_threshold=0.1, score_threshold=float('-inf')
        )
        horiz_lines = np.sort(np.array(horiz_out))

        im_nms = image_cv.copy()

        for val in horiz_lines:
            cv2.rectangle(im_nms, (int(horiz_boxes[val][0]), int(horiz_boxes[val][1])),
                          (int(horiz_boxes[val][2]), int(horiz_boxes[val][3])), (0, 0, 255), 1)

        # Apply Non-Maximum Suppression (NMS) for vertical boxes
        vert_out = tf.image.non_max_suppression(
            vert_boxes, probabilities, max_output_size=1000, iou_threshold=0.1, score_threshold=float('-inf')
        )
        vert_lines = np.sort(np.array(vert_out))

        for val in vert_lines:
            cv2.rectangle(im_nms, (int(vert_boxes[val][0]), int(vert_boxes[val][1])),
                          (int(vert_boxes[val][2]), int(vert_boxes[val][3])), (255, 0, 0), 1)

        # Create an empty table structure
        out_array = [["" for _ in range(len(vert_lines))] for _ in range(len(horiz_lines))]

        # Sort bounding boxes based on vertical position
        unordered_boxes = [vert_boxes[i][0] for i in vert_lines]
        ordered_boxes = np.argsort(unordered_boxes)

        # Fill the table using intersection and IoU logic
        for i in range(len(horiz_lines)):
            for j in range(len(vert_lines)):
                resultant = intersection(horiz_boxes[horiz_lines[i]], vert_boxes[vert_lines[ordered_boxes[j]]])

                for b in range(len(boxes)):
                    the_box = [boxes[b][0][0], boxes[b][0][1], boxes[b][2][0], boxes[b][2][1]]
                    if iou(resultant, the_box) > 0.1:
                        out_array[i][j] = texts[b]

        # Convert to a structured array
        out_array = np.array(out_array)

        # Save extracted text and structure as a CSV file with the image filename
        csv_filename = f"{os.path.splitext(image_file)[0]}.csv"
        csv_output_path = os.path.join(output_folder, csv_filename)
        pd.DataFrame(out_array).to_csv(csv_output_path, index=False, header=False)

        print(f"Processing completed for {image_file}. Results saved in {output_folder}")

    print("\n All images processed successfully! Extracted tables saved in:", output_folder)

input_directory = "PDF1 Cropped Tables"
output_directory = "PDF1 Tables 1"

extract_tables_from_images(input_directory, output_directory)